In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as func

In [2]:
spark = SparkSession.builder.getOrCreate()

In [3]:
genomes = spark.read.format('csv').option('header', 'true').load('genome_scores.csv')

In [4]:
genomes.show(3)

+-------+-----+---------+
|movieId|tagId|relevance|
+-------+-----+---------+
|      1|    1|    0.025|
|      1|    2|    0.025|
|      1|    3|  0.05775|
+-------+-----+---------+
only showing top 3 rows



In [5]:
ratings = spark.read.format('csv').option('header', 'true').load('rating.csv')

In [6]:
ratings.show(3)

+------+-------+------+-------------------+
|userId|movieId|rating|          timestamp|
+------+-------+------+-------------------+
|     1|      2|   3.5|2005-04-02 23:53:47|
|     1|     29|   3.5|2005-04-02 23:31:16|
|     1|     32|   3.5|2005-04-02 23:33:39|
+------+-------+------+-------------------+
only showing top 3 rows



### Aqui é onde o pandas falha, pois não consegue dar um merge nos dois arquivos e o spark consegue

In [7]:
merged = genomes.join(ratings, genomes.movieId == ratings.movieId).select(ratings.movieId,
                                                                          ratings.userId, 
                                                                          ratings.rating,
                                                                          genomes.tagId,
                                                                          genomes.relevance)

No comando anterior, o spark de fato não executou nada, apenas guardou a operações de merge na memória. Com o comando abaixo, em que estamos puxando as 3 primeiras linhas do dataset "merged", é que o pyspark vai realmente aplicar a operação, distribuindo o processamento pelos threads do processador.

In [8]:
merged.take(3)

[Row(movieId='100553', userId='1705', rating='4', tagId='1', relevance='0.018'),
 Row(movieId='100553', userId='11097', rating='4.5', tagId='1', relevance='0.018'),
 Row(movieId='100553', userId='14449', rating='4', tagId='1', relevance='0.018')]

In [38]:
merged.filter(merged.movieId=='100553').groupBy("movieId").agg(func.mean("rating").alias("mean rating")).take(3)

[Row(movieId='100553', mean rating=4.209677419354839)]

In [40]:
merged.groupBy("movieId").agg(func.mean("rating").alias("mean rating")).count()

10370

In [31]:
#merged.write.format('csv').option('header', 'true').option('delimiter',';').save('merged.csv')